In [1]:
import os

# 获取当前工作目录
current_dir = os.getcwd()
print("当前工作目录：", current_dir)

# 切换到上一层目录
parent_dir = os.path.dirname(current_dir)
os.chdir(parent_dir)
print("切换后的目录：", parent_dir)

当前工作目录： c:\Users\zhaoxs3\Documents\GitHub\NEMESIS\unit_test
切换后的目录： c:\Users\zhaoxs3\Documents\GitHub\NEMESIS


In [2]:
import QuantLib as ql
import pandas as pd

from devlib.products.fx import fx_vanilla
from devlib.market.curves.shibor import Shibor3M
from devlib.market.curves.fx_curves import FxForwardCurve, FxImpliedAssetCurve
from devlib.market.volatility.fx_vol_surface import FxVolSurfaceHK

from devlib.utils.ql_date_utils import ql_date, ql_date_str
from devlib.utils.fx_utils import get_calendar

In [3]:
d_ccy = 'CNY'
f_ccy = 'USD'
calendar = get_calendar(f_ccy, d_ccy)

today = ql.Date(25,3,2024)
ql.Settings.instance().evaluationDate = today
spot_f_d = 7.21145

# FX Forward Curve
forward_daycount = ql.Actual365Fixed()
forward_calendar = get_calendar(f_ccy, d_ccy)
forward_curve_file = './unit_test/data/USDCNY_curve_data_20240325.xlsx'
forward_curve_data = pd.read_excel(forward_curve_file)
forward_curve_data['SettleDate'] = forward_curve_data['SettleDate'].apply(lambda x: ql_date_str(ql_date(x)))
forward_curve = FxForwardCurve(today, spot_f_d, forward_curve_data, f_ccy, d_ccy, forward_calendar, forward_daycount)

# Domestic Discount Curve
d_curve_file ='./unit_test/data/shibor3m_curve_data_20240325.xlsx'
deposit_mkt_data = pd.read_excel(d_curve_file, sheet_name='deposit')
swap_mkt_data = pd.read_excel(d_curve_file, sheet_name='swap')
fixing_data = pd.read_excel(d_curve_file, sheet_name='fixing')
d_curve = Shibor3M(today, deposit_mkt_data=deposit_mkt_data, swap_mkt_data=swap_mkt_data, fixing_data=fixing_data)

# Foreign Implied Discount Curve
f_curve = FxImpliedAssetCurve(today, d_curve, forward_curve, calendar, daycount=ql.Actual365Fixed(), interpolation_method="zerolinear", tweak=0.0)

# FX Vol Surface
vol_surface_file = './unit_test/data/USDCNY_vol_data_20240325.xlsx'
vol_surface_data = pd.read_excel(vol_surface_file)
vol_surface = FxVolSurfaceHK(today, vol_surface_data, spot_f_d, d_ccy, f_ccy, forward_curve, d_curve, f_curve, forward_calendar, ql.Actual365Fixed())

In [4]:
expiry = ql.Date(17,1,2025)
payment_date = calendar.advance(expiry, ql.Period(2, ql.Days))
flavor = 'call'
strike = 7.3
notional = 1e6
notional_ccy = 'USD'
trade_direction = 'long'

inst = fx_vanilla.FxVanilla(d_ccy, f_ccy, calendar, expiry, payment_date, flavor, strike, notional, notional_ccy, trade_direction)

In [5]:
inst.npv(today, forward_curve, d_curve, vol_surface, daycount=ql.Actual365Fixed())

np.float64(35229.13215845862)

In [6]:
inst.delta(today, forward_curve, d_curve, vol_surface, daycount=ql.Actual365Fixed())

np.float64(184880.39339441457)

In [7]:
inst.gamma(today, forward_curve, d_curve, vol_surface, daycount=ql.Actual365Fixed())

np.float64(743565.7025780529)

In [8]:
inst.vega(today, forward_curve, d_curve, vol_surface, daycount=ql.Actual365Fixed())

np.float64(16890.474111567284)

In [9]:
inst.theta(today, forward_curve, d_curve, vol_surface, daycount=ql.Actual365Fixed())

np.float64(-152.63600804157613)

In [10]:
inst.rho(today, forward_curve, d_curve, vol_surface, daycount=ql.Actual365Fixed(), tweak_type="pillar_rate")

np.float64(107.75404724886903)

In [11]:
inst.rho(today, forward_curve, d_curve, vol_surface, daycount=ql.Actual365Fixed(), tweak_type="market")

np.float64(140.36166306813539)

In [12]:
inst.phi(today, forward_curve, d_curve, vol_surface, daycount=ql.Actual365Fixed(), tweak_type="pillar_rate")

np.float64(-110.67855887295445)

In [13]:
inst.phi(today, forward_curve, d_curve, vol_surface, daycount=ql.Actual365Fixed(), tweak_type="market")

np.float64(-144.1857904915778)

In [14]:
from nemesis.products.fx.fx_forward_curve import FXForwardCurve
from nemesis.products.fx.fx_implied_curve import FXImpliedAssetCurve
from nemesis.products.fx.fx_vol_surface import FXVolSurface
from nemesis.utils import *
from nemesis.products.rates import *
from nemesis.market.curves import InterpTypes
from nemesis.utils.calendar import JointCalendar

####################################################################
#  NEMESIS ALPHA Version 0.1.0 - This build: 24 Jan 2025 at 10:42 #
####################################################################



In [15]:
value_dt = Date(25, 3, 2024)
spot_fx_rate = 7.21145
currency_pair = "USDCNY"

fx_forward_curve = FXForwardCurve(value_dt, spot_fx_rate, forward_curve_data, currency_pair, JointCalendar([CalendarTypes.CHINA, CalendarTypes.UNITED_STATES]), DayCountTypes.ACT_365F, InterpTypes.FLAT_FWD_RATES)
domestic_curve = QLCurve(value_dt, d_curve, dc_type=DayCountTypes.ACT_360, interp_type=InterpTypes.LINEAR_ZERO_RATES)
foreign_implied_curve = FXImpliedAssetCurve(value_dt, domestic_curve, fx_forward_curve, JointCalendar([CalendarTypes.CHINA, CalendarTypes.UNITED_STATES]), DayCountTypes.ACT_365F, InterpTypes.LINEAR_ZERO_RATES)
fx_vol_surface = FXVolSurface(value_dt, vol_surface_data, spot_f_d, currency_pair, fx_forward_curve, foreign_implied_curve, cal_type=JointCalendar([CalendarTypes.CHINA, CalendarTypes.UNITED_STATES]), dc_type=DayCountTypes.ACT_365F)

In [16]:
from nemesis.products.fx.fx_vanilla_option import FXVanillaOption

In [17]:
expiry_dt = Date(17, 1, 2025)
strike_fx_rate = 7.3
notional = 1e6
prem_currency = "USD"

o = FXVanillaOption(expiry_dt, strike_fx_rate, currency_pair, OptionTypes.EUROPEAN_CALL, notional, prem_currency, JointCalendar([CalendarTypes.CHINA, CalendarTypes.UNITED_STATES]), spot_days=2)

In [18]:
o.value(value_dt, fx_forward_curve, domestic_curve, fx_vol_surface, DayCountTypes.ACT_365F)

{'value': np.float64(0.035229132158459055),
 'cash_dom': np.float64(35229.132158459055),
 'cash_for': np.float64(4885.166250678997),
 'pips_dom': np.float64(0.035229132158459055),
 'pips_for': np.float64(0.0006692008562573969),
 'pct_dom': np.float64(0.004825908514857405),
 'pct_for': np.float64(0.004885166250678997),
 'not_dom': 7300000.0,
 'not_for': 1000000.0,
 'ccy_dom': 'CNY',
 'ccy_for': 'USD'}

In [19]:
o.delta(value_dt, fx_forward_curve, domestic_curve, fx_vol_surface, DayCountTypes.ACT_365F) * notional

np.float64(184880.39339441957)

In [20]:
o.gamma(value_dt, fx_forward_curve, domestic_curve, fx_vol_surface, DayCountTypes.ACT_365F) * notional

np.float64(743565.6153298442)

In [21]:
o.vega(value_dt, fx_forward_curve, domestic_curve, fx_vol_surface, DayCountTypes.ACT_365F) * notional

np.float64(16890.47411156728)

In [22]:
o.theta(value_dt, fx_forward_curve, domestic_curve, fx_vol_surface, DayCountTypes.ACT_365F) * notional

np.float64(-152.63600804244632)

In [23]:
o.rho(value_dt, fx_forward_curve, domestic_curve, fx_vol_surface, DayCountTypes.ACT_365F, bump_type="pillar") * notional

np.float64(107.754047248651)

In [24]:
o.rho(value_dt, fx_forward_curve, domestic_curve, fx_vol_surface, DayCountTypes.ACT_365F, bump_type="market") * notional

np.float64(140.36166306780757)

In [25]:
o.phi(value_dt, fx_forward_curve, domestic_curve, fx_vol_surface, DayCountTypes.ACT_365F, bump_type="pillar") * notional

np.float64(110.67855887273923)

In [26]:
o.phi(value_dt, fx_forward_curve, domestic_curve, fx_vol_surface, DayCountTypes.ACT_365F, bump_type="market") * notional

np.float64(144.18579049125384)